In [2]:
import os
import shutil
import random

source_folder = r'C:\Users\abudh\Desktop\CropWatch\DATASET\Non_Vegetation2'
destination_folder = r'C:\Users\abudh\Desktop\CropWatch\Platform1_D2\Large_NonVegetation'

number_of_images_to_select = 2100
percentage_of_images_to_select = None

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

all_files = os.listdir(source_folder)
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif'}
image_files = [f for f in all_files if os.path.splitext(f)[1].lower() in image_extensions]

if number_of_images_to_select is not None:
    if len(image_files) < number_of_images_to_select:
        raise ValueError(f"Not enough images in the source folder to select {number_of_images_to_select}.")
    selected_files = random.sample(image_files, number_of_images_to_select)
elif percentage_of_images_to_select is not None:
    if not (0 < percentage_of_images_to_select <= 1):
        raise ValueError("Percentage must be between 0 and 1.")
    number_of_images_to_select = int(len(image_files) * percentage_of_images_to_select)
    selected_files = random.sample(image_files, number_of_images_to_select)
else:
    raise ValueError("Either 'number_of_images_to_select' or 'percentage_of_images_to_select' must be specified.")

for file_name in selected_files:
    source_file = os.path.join(source_folder, file_name)
    destination_file = os.path.join(destination_folder, file_name)
    shutil.copy(source_file, destination_file)

print(f"Successfully copied {len(selected_files)} images to {destination_folder}.")


Successfully copied 2100 images to C:\Users\abudh\Desktop\CropWatch\Platform1_D2\Large_NonVegetation.


In [1]:
import os
from PIL import Image

def pad_image(image, target_width, target_height):
    """Pads the image to the target width and height."""
    width, height = image.size
    new_width = max(target_width, width)
    new_height = max(target_height, height)

    # Calculate padding
    pad_width = new_width - width
    pad_height = new_height - height
    left = pad_width // 2
    top = pad_height // 2

    # Create a new image with the target size and paste the original image on it
    new_image = Image.new("RGB", (new_width, new_height), (255, 255, 255))  # White background
    new_image.paste(image, (left, top))
    
    return new_image

def splice_image(image_path, tile_width, tile_height):
    """
    Splices the image into tiles of specified width and height.
    Pads the image first to ensure all tiles are of the correct size.

    Parameters:
    - image_path: str, the path to the image file
    - tile_width: int, the width of each tile
    - tile_height: int, the height of each tile

    Returns:
    - cropped_images: list of PIL images, the spliced image tiles
    - positions: list of tuples, the (left, upper) positions of each tile in the padded image
    """
    img = Image.open(image_path)  # Open the image file
    width, height = img.size  # Get the dimensions of the image

    # Calculate the size to which we should pad the image
    target_width = (width // tile_width + 1) * tile_width
    target_height = (height // tile_height + 1) * tile_height

    img = pad_image(img, target_width, target_height)

    width, height = img.size  # Updated dimensions after padding

    cropped_images = []
    positions = []

    # Calculate the number of tiles in each direction
    n_tiles_width = width // tile_width
    n_tiles_height = height // tile_height

    for i in range(n_tiles_height):
        for j in range(n_tiles_width):
            left = j * tile_width
            upper = i * tile_height
            right = left + tile_width
            lower = upper + tile_height
            cropped_img = img.crop((left, upper, right, lower))  # Crop the image to create a tile
            cropped_images.append(cropped_img)  # Append the tile to the list
            positions.append((left, upper))
            
    return cropped_images, positions

def save_slices_to_folder(img_folder, cropped_img_folder, tile_width, tile_height):
    """
    Function to slice images into tiles and save them to the specified folder.

    Parameters:
    - img_folder: str, the folder containing the original images
    - cropped_img_folder: str, the folder where the cropped images will be saved
    - tile_width: int, the width of each tile
    - tile_height: int, the height of each tile
    """
    
    # Ensure the output directory exists
    if not os.path.exists(cropped_img_folder):
        os.makedirs(cropped_img_folder)
    
    # List all files in the input directory
    imgs = os.listdir(img_folder)

    for img_name in imgs:
        img_path = os.path.join(img_folder, img_name)
        
        # Process only image files
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            cropped_images, positions = splice_image(img_path, tile_width, tile_height)
            
            # Iterate over the cropped images with their index
            for idx, cropped_img in enumerate(cropped_images):
                # Save the slice with the correct file name
                left, upper = positions[idx]
                file_name, file_extension = os.path.splitext(img_name)
                cropped_img.save(os.path.join(cropped_img_folder, f'{file_name}_x{left}_y{upper}.jpeg'))
                
                print(f'Saved: {file_name}_x{left}_y{upper}.jpeg')
                print('-------------------------')
        else:
            print(f'Skipped non-image file: {img_name}')

# Folder paths
cropped_img_folder = r"C:\Users\abudh\Desktop\New folder2222"  # Destination folder for cropped images
img_folder = r"C:\Users\abudh\Desktop\CropWatch\DATASET\Large_Data"  # Source folder for original images

# Tile dimensions (224x224)
tile_width = 540
tile_height = 405

# Slice and save images
save_slices_to_folder(img_folder, cropped_img_folder, tile_width, tile_height)


Saved: 1694762942.7725067_camera_image_18_10_x0_y0.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x540_y0.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x1080_y0.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x1620_y0.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x2160_y0.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x0_y405.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x540_y405.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x1080_y405.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x1620_y405.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x2160_y405.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x0_y810.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_18_10_x540_y810.jpeg
----

In [5]:
from PIL import Image
import os

def get_corner_tiles(image, tile_width, tile_height):
    """Extracts tiles from the corners of the image."""
    width, height = image.size

    # Define the positions for the corners
    positions = [
        (0, 0),  # Top-left
        (width - tile_width, 0),  # Top-right
        (0, height - tile_height),  # Bottom-left
        (width - tile_width, height - tile_height)  # Bottom-right
    ]

    tiles = []

    for pos in positions:
        left, top = pos
        right = left + tile_width
        bottom = top + tile_height
        # Ensure the tile does not go out of bounds
        right = min(right, width)
        bottom = min(bottom, height)
        tile = image.crop((left, top, right, bottom))
        tiles.append(tile)

    return tiles

def save_corners_to_folder(img_folder, cropped_img_folder, tile_width, tile_height):
    """Saves corner tiles of images from a folder to a specified output folder."""
    
    # Ensure the output directory exists
    if not os.path.exists(cropped_img_folder):
        os.makedirs(cropped_img_folder)
    
    # List all files in the input directory
    imgs = os.listdir(img_folder)

    for img_name in imgs:
        img_path = os.path.join(img_folder, img_name)
        
        # Process only image files
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img = Image.open(img_path)
            corner_tiles = get_corner_tiles(img, tile_width, tile_height)
            
            # Iterate over the tiles with their index
            for idx, tile in enumerate(corner_tiles):
                # Save the slice with the correct file name
                file_name, file_extension = os.path.splitext(img_name)
                tile.save(os.path.join(cropped_img_folder, f'{file_name}_corner_{idx}.jpeg'))
                
                print(f'Saved: {file_name}_corner_{idx}.jpeg')
                print('-------------------------')
        else:
            print(f'Skipped non-image file: {img_name}')

# Folder paths
cropped_img_folder = r"C:\Users\abudh\Desktop\CropWatch\Filtered_Image_Platform1"  # Destination folder for cropped images
img_folder = r"C:\Users\abudh\Desktop\CropWatch\Test_15_08"  # Source folder for original images

# Tile dimensions (224x224)
tile_width = 224
tile_height = 224

# Slice and save corner tiles
save_corners_to_folder(img_folder, cropped_img_folder, tile_width, tile_height)


Saved: 1694762942.7725067_camera_image_10_11_corner_0.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_10_11_corner_1.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_10_11_corner_2.jpeg
-------------------------
Saved: 1694762942.7725067_camera_image_10_11_corner_3.jpeg
-------------------------
Saved: 1694798462.9411376_camera_image_9_11_corner_0.jpeg
-------------------------
Saved: 1694798462.9411376_camera_image_9_11_corner_1.jpeg
-------------------------
Saved: 1694798462.9411376_camera_image_9_11_corner_2.jpeg
-------------------------
Saved: 1694798462.9411376_camera_image_9_11_corner_3.jpeg
-------------------------
Saved: 1695132194.7972367_camera_image_7_6._20_corner_0.jpeg
-------------------------
Saved: 1695132194.7972367_camera_image_7_6._20_corner_1.jpeg
-------------------------
Saved: 1695132194.7972367_camera_image_7_6._20_corner_2.jpeg
-------------------------
Saved: 1695132194.7972367_camera_image_7_6._20_corner_3.jpeg
--

In [2]:
from PIL import Image
import os

# Folder paths
cropped_img_folder = r"C:\Users\abudh\Desktop\CropWatch\Filtered_Image_Platform1"  # Destination folder for cropped images
img_folder = r"C:\Users\abudh\Desktop\CropWatch\Test_15_08"  # Source folder for original images

# Create the destination folder if it does not exist
if not os.path.exists(cropped_img_folder):
    os.makedirs(cropped_img_folder)

# Process each image in the source folder
for filename in os.listdir(img_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Open an image file
        with Image.open(os.path.join(img_folder, filename)) as img:
            # Ensure the image is in RGB mode
            img = img.convert("RGB")
            
            # Image dimensions
            img_width, img_height = img.size
            print(f"Processing {filename}: {img_width}x{img_height}")
            
            # Define the size of the splices
            crop_width = img_width // 2
            crop_height = img_height // 2
            
            # Define the new size after resizing
            new_size = (224, 224)
            
            # Generate and save the splices
            for i in range(2):
                for j in range(2):
                    left = j * crop_width
                    upper = i * crop_height
                    right = left + crop_width
                    lower = upper + crop_height
                    
                    # Crop the image
                    cropped_img = img.crop((left, upper, right, lower))
                    
                    # Resize the cropped image
                    resized_img = cropped_img.resize(new_size, Image.LANCZOS)
                    
                    # Save the resized image
                    cropped_img_filename = f"{os.path.splitext(filename)[0]}_crop{i*2+j+1}.jpg"
                    resized_img.save(os.path.join(cropped_img_folder, cropped_img_filename))
                    
                    print(f"Saved {cropped_img_filename}")

print("Processing completed.")


Processing 1695132194.7972367_camera_image_7_6._20.jpeg: 540x405
Saved 1695132194.7972367_camera_image_7_6._20_crop1.jpg
Saved 1695132194.7972367_camera_image_7_6._20_crop2.jpg
Saved 1695132194.7972367_camera_image_7_6._20_crop3.jpg
Saved 1695132194.7972367_camera_image_7_6._20_crop4.jpg
Processing 1697285856.3967595_camera_image_14_1._10._00.jpeg: 540x405
Saved 1697285856.3967595_camera_image_14_1._10._00_crop1.jpg
Saved 1697285856.3967595_camera_image_14_1._10._00_crop2.jpg
Saved 1697285856.3967595_camera_image_14_1._10._00_crop3.jpg
Saved 1697285856.3967595_camera_image_14_1._10._00_crop4.jpg
Processing 1699523874.8845115_camera_image_20_4._11._00.jpeg: 540x405
Saved 1699523874.8845115_camera_image_20_4._11._00_crop1.jpg
Saved 1699523874.8845115_camera_image_20_4._11._00_crop2.jpg
Saved 1699523874.8845115_camera_image_20_4._11._00_crop3.jpg
Saved 1699523874.8845115_camera_image_20_4._11._00_crop4.jpg
Processing 1704536206.8365126_camera_image_18_7._11.jpeg: 540x405
Saved 1704536206.